# Stochastic spatio-temporal growth model

In [34]:
import numpy as np
import pandas as pd
from scipy.optimize import fsolve

## Simulation

### Three-value model
$$
g = \begin{cases}
1/2~~\text{with probability}~~p_{1/2}\\
1~~\text{with probability}~~p_{1}\\
2~~\text{with probability}~~p_{2}
\end{cases}
$$

In [11]:
# Parameters
dp = 1000000
alpha = 0.7
p_down = 0.2
p_up = p_down * (1 - 2**(-alpha)) / (2**alpha - 1)
p_stay = 1 - p_down - p_up
values = [1/2, 1, 2]
probabilities = [p_down, p_stay, p_up]
M = 1000  # spatial corse-grainig parameter
tau = 30  # temporal corse-grainig parameter
S = 1000  # number of species
c = 1  # external force
seed = 0
T = dp * tau

# Set random seed
rng = np.random.default_rng(seed=seed)

# Initialize variables
current = rng.integers(1, 10, size=(S, M)) # initial abundance at each site for each species
N = np.zeros((S, dp)) # time series of coarse-grained abundance for each species
N[:, 0] = np.sum(current, axis=1) # initial coarse-grained abundance for each species

# Time evolution loop
for t in range(1, dp):
    # Sample growth rates and calculate abundance
    g = rng.choice(values, size=(S, M, tau), p=probabilities)
    current = current * np.prod(g, axis=2) + c * np.sum(np.cumprod(g[:, :, 1:][:, :, ::-1], axis=2), axis=2) + c
    N[:, t] = np.sum(current, axis=1)

# Compute growth rates
G = N[:, 1:] / N[:, :-1]

### Log-uniform model
$$
p(g) = \begin{cases}
\frac{1}{g\cdot \ln \left( g_{max}/g_{min}\right)}~~\text{for}~~g_{min}\leq g \leq g_{max}\\
0~~\text{otherwise}
\end{cases}
$$

In [33]:
def equation(b):
    return (np.exp((b+e)*alpha)-np.exp((-b+e)*alpha))/(2*b*alpha) - 1

def log_uniform(low, high, size):
    return np.exp(np.random.uniform(np.log(low), np.log(high), size))

# Parameters
dp = 1000000
alpha = 0.7
e = -0.1
initial_guess = 1.0
b = fsolve(equation, initial_guess)[0]
low = np.exp(-b + e)  # lower bound
high = np.exp(b + e)  # upper bound
M = 1000  # spatial corse-grainig parameter
tau = 30  # temporal corse-grainig parameter
S = 1000  # number of species
c = 1  # external force
seed = 0
T = dp * tau

# Set random seed
rng = np.random.default_rng(seed=seed)

# Initialize variables
current = rng.integers(1, 10, size=(S, M)) # initial abundance at each site for each species
N = np.zeros((S, dp)) # time series of coarse-grained abundance for each species
N[:, 0] = np.sum(current, axis=1) # initial coarse-grained abundance for each species

# Time evolution loop
for t in range(1, dp):
    # Sample growth rates and calculate abundance
    g = log_uniform(low, high, size=(S, M, tau))
    current = current * np.prod(g, axis=2) + c * np.sum(np.cumprod(g[:, :, 1:][:, :, ::-1], axis=2), axis=2) + c
    N[:, t] = np.sum(current, axis=1)

# Compute growth rates
G = N[:, 1:] / N[:, :-1]

### Binary model
$$
g = \begin{cases}
g_-~~\text{with probability}~~1/2\\
g_+~~\text{with probability}~~1/2\\
\end{cases}
$$

In [18]:
# Parameters
dp = 1000000
alpha = 0.7
p0 = 1/2
p1 = 1/2
gamma = 3
g0 = (2/(1+gamma**alpha))**(1/alpha)
g1 = gamma*g0
values = [g0, g1]
probabilities=[p0, p1]
M = 1000  # spatial corse-grainig parameter
tau = 30  # temporal corse-grainig parameter
S = 1000  # number of species
c = 1  # external force
seed = 0
T = dp * tau

# Set random seed
rng = np.random.default_rng(seed=seed)

# Initialize variables
current = rng.integers(1, 10, size=(S, M)) # initial abundance at each site for each species
N = np.zeros((S, dp)) # time series of coarse-grained abundance for each species
N[:, 0] = np.sum(current, axis=1) # initial coarse-grained abundance for each species

# Time evolution loop
for t in range(1, dp):
    # Sample growth rates and calculate abundance
    g = rng.choice(values, size=(S, M, tau), p=probabilities)
    current = current * np.prod(g, axis=2) + c * np.sum(np.cumprod(g[:, :, 1:][:, :, ::-1], axis=2), axis=2) + c
    N[:, t] = np.sum(current, axis=1)

# Compute growth rates
G = N[:, 1:] / N[:, :-1]

## Spatial diffusion

In [42]:
# Parameters
dp = 3
alpha = 0.7
L = 100  # Grid size
# Three-value model
p0 = 0.2
p2 = p0 * (1 - 2**(-alpha)) / (2**alpha - 1)
p1 = 1 - p0 - p2
values = [1/2, 1, 2]
probabilities = [p0, p1, p2]
c = 1  # External force
d = 0.0001  # Diffusion coefficients
seed = 0

# Set random seed
rng = np.random.default_rng(seed)

# Initial values
allxs = np.zeros((L * L, dp))
allxs[:, 0] = np.random.randint(1,10,size=L*L)

# Time evolution with or without diffusion
for t in range(1, dp):
    g = rng.choice(values, size=L * L, p=probabilities)
    for i in range(L):
        for j in range(L):
            # Neighboring cells with periodic boundary conditions
            ai, bj, ci, dj = i-1, j+1, i+1, j-1
            ai, ci = ai % L, ci % L
            bj, dj = bj % L, dj % L

            # Update cell value with diffusion term
            ind = i * L + j
            ind_a, ind_b, ind_c, ind_d = ai * L + j, i * L + bj, ci * L + j, i * L + dj
            allxs[ind, t] = ((1 - 4 * d) * allxs[ind, t - 1] + d * (allxs[ind_a, t - 1] + allxs[ind_b, t - 1] + allxs[ind_c, t - 1] + allxs[ind_d, t - 1])) * g[ind] + c
